In [22]:
dir_api = '/Users/jnaiman/Dropbox/JCDL2024/paper_submission/chatgpt_api/'

# where to save jsons
dir_api_jsons = '/Users/jnaiman/Dropbox/JCDL2024/paper_submission/chatgpt_api_jsons/'

In [181]:
from glob import glob
import pickle
import json
import numpy as np
import pandas as pd

In [24]:
files = glob(dir_api+'*.pickle')

In [25]:
files[:5], len(files)

(['/Users/jnaiman/Dropbox/JCDL2024/paper_submission/chatgpt_api/Picture182.pickle',
  '/Users/jnaiman/Dropbox/JCDL2024/paper_submission/chatgpt_api/Picture72.pickle',
  '/Users/jnaiman/Dropbox/JCDL2024/paper_submission/chatgpt_api/Picture60.pickle',
  '/Users/jnaiman/Dropbox/JCDL2024/paper_submission/chatgpt_api/Picture1.pickle',
  '/Users/jnaiman/Dropbox/JCDL2024/paper_submission/chatgpt_api/Picture35.pickle'],
 107)

In [93]:
# dump in jsons
for file in files:
    with open(file, 'rb') as f:
        qa = pickle.load(f)[0]
    #import sys; sys.exit()
    with open(dir_api_jsons + file.split('/')[-1].split('.pickle')[0] + '.json','w') as f:
        json.dump(qa,f,indent=4)

In [104]:
known_files = ['/Users/jnaiman/Dropbox/JCDL2024/paper_submission/chatgpt_api_jsons/Picture169.json']
known_qs = ['What are the data values in this figure? You are a helpful assistant, please format the output as a json as {"x":[], "y":[], "colors":[]} where the values of "x", "y", and "colors" are the data values used to create the plot and are in the format of a list of floats.  ']


In [261]:
# Program to measure the similarity between  
# two sentences using cosine similarity. 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
  
# # X = input("Enter first string: ").lower() 
# # Y = input("Enter second string: ").lower() 
# X ="I love horror movies"
# Y ="Lights out is a horror movie"




In [233]:
questions = []
correct_answers = []
gpt_answers = []
levels = []
level_type = []
files_out = []


icount = 0
for file in files_json:
    with open(file,'r') as f:
        j = json.loads(f.read())

    print('')
    print('************************')

    # test
    for qa in j:
        err = False
        icount += 1
        ans = qa['A']
        if 'Response String' in qa and qa['Response'] == qa['Response String']:
            continue
        if type(qa['Response']) != type(ans):
            print('have mismatched')
            if file in known_files and qa['Q'] in known_qs: # known issue
                err = True
                #import sys; sys.exit()
            else:
                # lets try some stuff
                if type(qa['Response']) == type({}):
                    ans_1 = {}
                    ans_1[list(qa['Response'].keys())[0]] = ans
                    ans = ans_1
                    gpt = qa['Response'].copy()
                else:
                    print('error out on:', qa['Response'])
                    err = True
                    #import sys; sys.exit()
        elif type(qa['Response']) == type({}):
            gpt = qa['Response'].copy()
        else:
            gpt = qa['Response']

        if err:
            continue

        # check for ticks
        if 'xtick values' in ans:
            ans_1 = []
            for l in ans['xtick values']:
                aa = []
                for ll in l:
                    aa.append(float(ll.split("{")[-1].split('}')[0].replace('−','-')))
                ans_1.append(aa)
            ans['xtick values'] = ans_1
        if 'ytick values' in ans:
            ans_1 = []
            for l in ans['ytick values']:
                aa = []
                for ll in l:
                    aa.append(float(ll.split("{")[-1].split('}')[0].replace('−','-')))
                ans_1.append(aa)
            ans['ytick values'] = ans_1

        if ans != gpt:
            # see if we can reformat
            if type(ans) == type({}):
                if ans.keys() != gpt.keys(): # not the same keys
                    print('wrong keys')
                    err = True
                    print(ans, gpt)
                    #import sys; sys.exit()
                else:
                    for k,v in ans.items():
                        if k == 'aspect ratio': # try specific things
                            if ':' in gpt[k]:
                                v1 = float(gpt[k].split(':')[0])
                                v2 = float(gpt[k].split(':')[1])
                                gpt[k] = v1/v2
                        elif type(v) == int:
                            try:
                                gpt[k] = int(gpt[k])
                            except Exception as e2:
                                print(e2)
                                err = True
                        elif type(v) == float:
                            try:
                                gpt[k] = float(gpt[k])
                            except Exception as e3:
                                print(e3)
                                err = True
                # try again:
                if ans != gpt:
                    print('not same:', ans, gpt)
                    print('')
                    #if icount > 38: import sys; sys.exit()
            else:
                print('not dict')
                print(ans, gpt)
                err = True
                pass

        # save if not rror
        if not err:
            questions.append(qa['Q'])
            correct_answers.append(ans)
            gpt_answers.append(gpt)
            levels.append(qa['Level'])
            level_type.append(qa['type'])
            files_out.append(file.split('/')[-1])
            

df_qa = pd.DataFrame({'question':questions, 'correct answer':correct_answers, 'gpt answer':gpt_answers, 'level':levels, 'level type':level_type, 'file':files_out})
print('DONE!')


************************
not same: {'plot style': 'fast'} {'plot style': 'seaborn-whitegrid'}

not same: {'colormap': 'gray'} {'colormap': 'Blues'}

not same: {'aspect ratio': 0.7040921867890633} {'aspect ratio': 0.7092198581560284}

have mismatched
have mismatched
not same: {'npoints': 106} {'npoints': 77}

have mismatched
have mismatched
not same: {'scatter plot marker': 'P'} {'scatter plot marker': '.'}

have mismatched
not same: {'marker size': 1} {'marker size': 3}

have mismatched
not same: {'titles': ['']} {'titles': ['|dt|/|dz| SHELLS HOSPITALITY']}

have mismatched
not same: {'xlabels': ['Dissipation Probes $v_{\\rm BH} \\approx 1600$ ']} {'xlabels': ['Dissipation Probes $\\nu_{BH} \\approx 1600$']}

have mismatched
not same: {'xtick values': [[-5000.0, -2500.0, 0.0, 2500.0, 5000.0, 7500.0, 10000.0, 12500.0, 15000.0, 17500.0]]} {'xtick values': [[-2500, 0, 2500, 5000, 7500, 10000, 12500, 15000]]}

have mismatched
not same: {'ylabels': ['$|dt/dz|$  SHELLS HOSPITALITY']} {'ylab

In [469]:
df1 = df_qa[df_qa['file']=='Picture169.json']

In [478]:
for i in range(len(df1)):
    d = df1.iloc[i]
    print(d['level type'], d['level'])
    #print('')

Figure-level questions Level 1
Figure-level questions Level 1
Figure-level questions Level 1
Figure-level questions Level 1
Plot-level questions Level 1
Plot-level questions Level 1
Plot-level questions Level 1
Plot-level questions Level 1
Plot-level questions Level 1
Plot-level questions Level 1
Plot-level questions Level 1
Plot-level questions Level 1
Plot-level questions Level 1
Plot-level questions Level 2
Plot-level questions Level 2
Plot-level questions Level 3
Plot-level questions Level 3


In [222]:
df_qa['level'].unique()

array(['Level 1', 'Level 2', 'Level 3'], dtype=object)

In [234]:
df_qa

,question,correct answer,gpt answer,level,level type,file
0,How many panels are in this figure? You are a ...,"{'nrows': 1, 'ncols': 1}","{'nrows': 1, 'ncols': 1}",Level 1,Figure-level questions,Picture169.json
1,Assuming this is a figure made with matplotlib...,{'plot style': 'fast'},{'plot style': 'seaborn-whitegrid'},Level 1,Figure-level questions,Picture169.json
2,Assuming this is a figure made with matplotlib...,{'colormap': 'gray'},{'colormap': 'Blues'},Level 1,Figure-level questions,Picture169.json
3,What is the aspect ratio of this figure? You a...,{'aspect ratio': 0.7040921867890633},{'aspect ratio': 0.7092198581560284},Level 1,Figure-level questions,Picture169.json
4,How many scatter points are there for on the f...,{'npoints': 106},{'npoints': 77},Level 1,Plot-level questions,Picture169.json
...,...,...,...,...,...,...
2123,What are the y-axis titles for each figure pan...,"{'ylabels': ['$v_0(z,t)$ Areas Words']}","{'ylabels': ['$u(t, z) \text{ Areas Words}$']}",Level 1,Plot-level questions,Picture79.json
2124,What are the values for each of the tick marks...,"{'ytick values': [[0.0, 0.5, 1.0, 1.5, 2.0, 2....","{'ytick values': [[0.0, 0.5, 1.0, 1.5, 2.0, 2....",Level 1,Plot-level questions,Picture79.json
2125,What are the maximum data values in this figur...,{'maximum x': -6.677373136724738},{'maximum x': 1.0},Level 2,Plot-level questions,Picture79.json
2126,Are there error bars on the data along the y-a...,{'y-axis errors': True},{'y-axis errors': True},Level 2,Plot-level questions,Picture79.json


In [235]:
df_qa['file'].nunique()

105

In [398]:
def cosine_sim(X,Y):
    # tokenization 
    X_list = word_tokenize(X)  
    Y_list = word_tokenize(Y) 
      
    # # sw contains the list of stopwords 
    # sw = stopwords.words('english')  
    l1 =[];l2 =[] 
      
    # # remove stop words from the string 
    # X_set = {w for w in X_list if not w in sw}  
    # Y_set = {w for w in Y_list if not w in sw} 
    X_set = set(X_list)
    Y_set = set(Y_list)
      
    # form a set containing keywords of both strings  
    rvector = X_set.union(Y_set)  
    for w in rvector: 
        if w in X_set: l1.append(1) # create a vector 
        else: l1.append(0) 
        if w in Y_set: l2.append(1) 
        else: l2.append(0) 
    c = 0
      
    # cosine formula  
    for i in range(len(rvector)): 
            c+= l1[i]*l2[i] 
    cosine = c / float((sum(l1)*sum(l2))**0.5) 
    #print("similarity: ", cosine) 
    return cosine


import Levenshtein

In [401]:
for d in df_qa['question'].unique():
    if 'title' in d and 'color' not in d:
        print(repr(d))

'What are the titles for each figure panel? You are a helpful assistant, please format the output as a json as {"titles":[]}, where the list is a list of strings of all of the titles.  If there is a single plot, this should be one element in this list, and if there are multiple plots the list should be in row-major (C-style) order. If a plot does not have a title, then denote this by an empty string in the list.  Please format any formulas in the title in a Python LaTeX string (for example \'Light $\\\\alpha$\'). '
'What are the x-axis titles for each figure panel? You are a helpful assistant, please format the output as a json as {"xlabels":[]}, where the list is a list of strings of all of the x-axis titles.  If there is a single plot, this should be one element in this list, and if there are multiple plots the list should be in row-major (C-style) order. If a plot does not have an x-axis title, then denote this by an empty string in the list.  Please format any formulas in the title

In [227]:
npoints_scatter = []
for d in df_qa['question'].unique():
    if '"npoints"' in d:
        #print(d)
        npoints_scatter.append(d)


functional_xy = []
for d in df_qa['question'].unique():
    if ('functional relationship' in d or 'distribution used' in d) and 'color' not in d:
        functional_xy.append(d)

mean_xy = []
for d in df_qa['question'].unique():
    if 'mean y' in d and 'color' not in d:
        mean_xy.append(d)

median_xy = []
for d in df_qa['question'].unique():
    if 'median y' in d and 'color' not in d:
        median_xy.append(d)

minimum_xy = []
for d in df_qa['question'].unique():
    if 'minimum y' in d and 'color' not in d:
        minimum_xy.append(d)

maximum_xy = []
for d in df_qa['question'].unique():
    if 'maximum y' in d and 'color' not in d:
        maximum_xy.append(d)

In [447]:
# list of specific questions we want to have answered
questions = [
    {'Number of panels':'How many panels are in this figure? You are a helpful assistant, please format the output as a json as {"nrows":"", "ncols":""} to store the number of rows and columns.'},
    {'Plot style':'Assuming this is a figure made with matplotlib in Python, what is the plot style used?  Examples of plotting styles are "classic" or "ggplot". You are a helpful assistant, please format the output as a json as {"plot style":""} to store the matplotlib plotting style used in the figure.'},
    {'Colormap':'Assuming this is a figure made with matplotlib in Python, what is the colormap that was used?  Examples of matplotlib colormaps are "rainbow" or "Reds". You are a helpful assistant, please format the output as a json as {"colormap":""} to store the matplotlib colormap used in the figure.'},
    {'Aspect ratio':'What is the aspect ratio of this figure? You are a helpful assistant, please format the output as a json as {"aspect ratio":""} to store the aspect ratio of the plot.'},
    {'Plot type':'What are the plot types for each panel in the figure? You are a helpful assistant, please format the output as a json as {"plot types":[]}, where each element of the list refers to the plot type of a single panel. If there is a single plot, this should be one element in this list, and if there are multiple plots the list should be in row-major (C-style) order. Please choose each plot type from the following list: [line, scatter, histogram].'},
    {'Number of scatter points':npoints_scatter},
    {'Titles':'What are the titles for each figure panel? You are a helpful assistant, please format the output as a json as {"titles":[]}, where the list is a list of strings of all of the titles.  If there is a single plot, this should be one element in this list, and if there are multiple plots the list should be in row-major (C-style) order. If a plot does not have a title, then denote this by an empty string in the list.  Please format any formulas in the title in a Python LaTeX string (for example \'Light $\\\\alpha$\'). '},
    {'x-axis titles':'What are the x-axis titles for each figure panel? You are a helpful assistant, please format the output as a json as {"xlabels":[]}, where the list is a list of strings of all of the x-axis titles.  If there is a single plot, this should be one element in this list, and if there are multiple plots the list should be in row-major (C-style) order. If a plot does not have an x-axis title, then denote this by an empty string in the list.  Please format any formulas in the title in a Python LaTeX string (for example \'Light $\\\\alpha$\'). '},
    {'y-axis titles':'What are the y-axis titles for each figure panel? You are a helpful assistant, please format the output as a json as {"ylabels":[]}, where the list is a list of strings of all of the y-axis titles.  If there is a single plot, this should be one element in this list, and if there are multiple plots the list should be in row-major (C-style) order. If a plot does not have an y-axis title, then denote this by an empty string in the list.  Please format any formulas in the title in a Python LaTeX string (for example \'Light $\\\\alpha$\'). '},
    {'x-y relationship':functional_xy},
    {'mean x/y':mean_xy},
    {'median x/y':median_xy}
             ]

questions_one = questions.copy()

In [459]:
for q in questions:
    qt = list(q.keys())[0]
    #accuracy_this_q = 0
    accuracy_type = ''
    values_correct = []; values_gpt = []
    if type(q[qt]) == str: # just one question
        d = df_qa[df_qa['question'] == q[qt]]
        q['Level'] = d.iloc[0]['level']
        # what type
        if len(d) == 0:
            print('no matches for:', q[qt])
            continue
        if type(d.iloc[0]['correct answer']) == type({}):
            isNum = True
            for k,v in d.iloc[0]['correct answer'].items():
                try:
                    x = v/1.0
                except:
                    isNum = False
            if isNum: # number
                accuracy_type = 'Numerical'
                for c,gpt in zip(d['correct answer'].values, d['gpt answer'].values):
                    for k,v in c.items():
                        values_correct.append(float(v))
                        values_gpt.append(float(gpt[k]))

                #accuracy_this_q = np.sum(np.abs(np.array(values_correct)-np.array(values_gpt)))/np.sum(np.array(values_correct))
                accuracy_this_q = np.median(np.abs(np.array(values_correct)-np.array(values_gpt))/np.array(values_correct))
                q['Accuracy'] = accuracy_this_q
                q['Accuracy Type'] = 'Numerical'
                q['Correct Values'] = values_correct
                q['GPT Values'] = values_gpt
            else:
                #print('not num!')
                isStr = False
                for k,v in d.iloc[0]['correct answer'].items():
                    if type(v) == str:
                        isStr = True
                if isStr:  
                    #print('is string!')
                    #import sys; sys.exit()
                    accuracy_type = 'String'
                    vcos = []
                    vc = []; gv = []
                    for c,gpt in zip(d['correct answer'].values, d['gpt answer'].values):
                        for k,v in c.items():
                            #print(v,gpt[k],cosine_sim(v,gpt[k]))
                            #vcos.append(cosine_sim(v,gpt[k]))
                            e = Levenshtein.distance(v,gpt[k])
                            bigger = np.max([len(v),len(gpt)])
                            #print(v,gpt[k],np.abs(1.0-e/bigger))
                            vcos.append(np.abs(e/bigger))
                            vc.append(v); gv.append(gpt[k])

                    q['Accuracy'] = np.mean(vcos)
                    q['Accuracy Type'] = 'String'
                    q['Correct Values'] = vc
                    q['GPT Values'] = gv
                else:
                    # try list
                    isList = False
                    for k,v in d.iloc[0]['correct answer'].items():
                        if type(v) == list:
                            isList = True 
                    if isList:
                        #print('is list')
                        vcos = []
                        cos_type = ''
                        vc = []; gc = []
                        for c,gpt in zip(d['correct answer'].values, d['gpt answer'].values):
                            for k,v in c.items():
                                cor_vec = []
                                g_vec = []
                                isStr1 = False; isNum1 = False
                                for cc in v:
                                    cor_vec.append(cc)
                                    try:
                                        x = cc/1.0
                                        isNum1 = True
                                    except:
                                        pass
                                    if type(cc) == str:
                                        isStr1 = True
                                if isStr1 and isNum1:
                                    print('mixed types, ours!')
                                    continue

                                isStr1 = False; isNum1 = False
                                for cc in gpt[k]:
                                    g_vec.append(cc)
                                    try:
                                        x = cc/1.0
                                        isNum1 = True
                                    except:
                                        pass
                                    if type(cc) == str:
                                        isStr1 = True
                                if isStr1 and isNum1:
                                    print('mixed types, gpt!')
                                    continue

                                if isStr1:
                                    s1 = "".join(cor_vec)
                                    s2 = "".join(g_vec)
                                    e = Levenshtein.distance(s1,s2)
                                    bigger = np.max([len(s1),len(s2)])
                                    vcos.append(np.abs(e/bigger))   
                                    #print(s1,s2, e/bigger)
                                    if cos_type == '':
                                        cos_type = ['String']
                                    else: 
                                        cos_type.append('String')
                                    vc.append(s1); gc.append(s2)
                                else:
                                    print('not string array!')
                                    import sys; sys.exit()
                                
                                #cc = []; gg = []
                                #if len(v) != len(gpt[k]): # wrong items
                                #for cc,gg in zip(v,gpt[k]): # lists
                        u = np.unique(cos_type)
                        if len(u) > 1:
                            print('too many cos types!')
                            import sys; sys.exit()
                        else:
                            u = u[0]
                        q['Accuracy'] = np.mean(vcos)
                        q['Accuracy Type'] = u
                        q['Correct Values'] = vc
                        q['GPT Values'] = gc
                        #import sys; sys.exit()
                        #pass
                    else:
                            
                        print('not string or num or list!')
                        import sys; sys.exit()
        else:
            print('not dict!')
            import sys; sys.exit()
    else: # this is for things that have multiple question types
        #print('not a string!')
        qout = []
        print(qt)
        for qq in q[qt]:
            values_correct = []; values_gpt = []
            vc = []; gc = []
            d = df_qa[df_qa['question'] == qq]
            q['Level'] = d.iloc[0]['level']

            # what type
            if len(d) == 0:
                print('no matches for:', qq)
                continue
            if type(d.iloc[0]['correct answer']) == type({}):
                isNum = True
                for k,v in d.iloc[0]['correct answer'].items():
                    try:
                        x = v/1.0
                    except:
                        isNum = False
                if isNum: # number
                    accuracy_type = 'Numerical'
                    for c,gpt in zip(d['correct answer'].values, d['gpt answer'].values):
                        for k,v in c.items():
                            values_correct.append(float(v))
                            values_gpt.append(float(gpt[k]))
    
                    #accuracy_this_q = np.sum(np.abs(np.array(values_correct)-np.array(values_gpt)))/np.sum(np.array(values_correct))
                    accuracy_this_q = np.median(np.abs(np.array(values_correct)-np.array(values_gpt))/np.abs(np.array(values_correct)))
                    print(accuracy_this_q)
                    qout.append({'Accuracy': accuracy_this_q,'Accuracy Type': 'Numerical', 'Correct Values':values_correct, 'GPT Values':values_gpt})
                else:
                    #print('not num!')
                    isStr = False
                    for k,v in d.iloc[0]['correct answer'].items():
                        if type(v) == str:
                            isStr = True
                    if isStr:  
                        #print('is string!')
                        #import sys; sys.exit()
                        accuracy_type = 'String'
                        vcos = []
                        vc = [];gc = []
                        for c,gpt in zip(d['correct answer'].values, d['gpt answer'].values):
                            for k,v in c.items():
                                #print(v,gpt[k],cosine_sim(v,gpt[k]))
                                #vcos.append(cosine_sim(v,gpt[k]))
                                e = Levenshtein.distance(v,gpt[k])
                                bigger = np.max([len(v),len(gpt)])
                                #print(v,gpt[k],np.abs(1.0-e/bigger))
                                vcos.append(np.abs(e/bigger))
                                vc.append(v);gc.append(gpt[k])
    
                        qout.append({'Accuracy': np.mean(vcos),'Accuracy Type':'String', 'Correct Values':vc, 'GPT Values':gc})
                    else:
                        # try list
                        isList = False
                        for k,v in d.iloc[0]['correct answer'].items():
                            if type(v) == list:
                                isList = True 
                        if isList:
                            print('is list')
                            vcos = []
                            cos_type = ''
                            vc = []; gc = []
                            for c,gpt in zip(d['correct answer'].values, d['gpt answer'].values):
                                vvc = []; ggc =[]
                                for k,v in c.items():
                                    cor_vec = []
                                    g_vec = []
                                    isStr1 = False; isNum1 = False
                                    for cc in v:
                                        cor_vec.append(cc)
                                        try:
                                            x = cc/1.0
                                            isNum1 = True
                                        except:
                                            pass
                                        if type(cc) == str:
                                            isStr1 = True
                                    if isStr1 and isNum1:
                                        print('mixed types, ours!')
                                        continue
    
                                    isStr1 = False; isNum1 = False
                                    for cc in gpt[k]:
                                        g_vec.append(cc)
                                        try:
                                            x = cc/1.0
                                            isNum1 = True
                                        except:
                                            pass
                                        if type(cc) == str:
                                            isStr1 = True
                                    if isStr1 and isNum1:
                                        print('mixed types, gpt!')
                                        continue
    
                                    if isStr1:
                                        s1 = "".join(cor_vec)
                                        s2 = "".join(g_vec)
                                        #print(s1,s2)
                                        vvc.append(s1); ggc.append(s2)
                                        e = Levenshtein.distance(s1,s2)
                                        bigger = np.max([len(s1),len(s2)])
                                        vcos.append(np.abs(e/bigger))   
                                        #print(s1,s2, e/bigger)
                                        if cos_type == '':
                                            cos_type = ['String']
                                        else: 
                                            cos_type.append('String')
                                        vvc.append(s1);ggc.append(s2)
                                    elif isNum1:
                                        try:
                                            vcos.append(np.mean(np.abs(np.array(cor_vec)-np.array(g_vec))/np.abs(np.array(cor_vec))))
                                        except:
                                            vcos.append(np.mean(np.abs(np.mean(cor_vec)-np.mean(g_vec))/np.abs(np.array(cor_vec))))

                                        vvc.append(cor_vec); ggc.append(g_vec)
                                        if cos_type == '':
                                            cos_type = ['Numerical']
                                        else: 
                                            cos_type.append('Numerical')

                                    else:
                                        print('not string or num array!')
                                        import sys; sys.exit()
                                    
                                    #cc = []; gg = []
                                    #if len(v) != len(gpt[k]): # wrong items
                                    #for cc,gg in zip(v,gpt[k]): # lists
                                vc.append(vvc); gc.append(ggc)
                            u = np.unique(cos_type)
                            if len(u) > 1:
                                print('too many cos types!')
                                import sys; sys.exit()
                            else:
                                u = u[0]
                            print(np.mean(vcos))
                            qout.append({'Accuracy':np.mean(vcos),'Accuracy Type':u, 'Correct Values':vc, 'GPT Values':gc})
                            #import sys; sys.exit()
                            #pass
                        else:
                                
                            print('not string or num or list!')
                            import sys; sys.exit()
            else:
                print('not dict!')
                import sys; sys.exit()

        atypes = []
        accs = []
        vc = []; gc = []
        for qq in qout:
            atypes.append(qq['Accuracy Type'])
            accs.append(qq['Accuracy'])
            vc.append(qq['Correct Values'])
            gc.append(qq['GPT Values'])
        u = np.unique(atypes)
        if len(u) > 1:
            print('mixed acc types!')
            import sys; sys.exit()
        else:
            u = u[0]
        q['Accuracy Type'] = u
        q['Accuracy'] = np.median(accs)
        #print(np.mean(accs))
        q['Correct Values'] = vc
        q['GPT Values'] = gc
        #import sys; sys.exit()


Number of scatter points
0.3918918918918919
0.9504132231404959
0.62
0.7715678919729932
0.5876288659793815
1.0
0.8113888888888889
0.7338709677419355
1.0
0.7375
0.05128205128205128
0.9411764705882353
1.9411764705882353
x-y relationship
is list
0.6243536476083018
is list
0.0
is list
0.4356060606060606
is list
0.725
is list
0.5982954545454545
is list
0.625
is list
0.625
is list
0.8376623376623377
is list
0.7777777777777778
is list
0.5285714285714286
is list
0.775
mean x/y
is list
0.8738000370049
is list
1.9636634843485348
is list
0.9331349913513894
is list
409.3856299942949
is list
1.011438849354316
is list
0.8433553545490938
median x/y
is list
0.8767083056496962
is list
1.9674159571914036
is list
0.9911088079448755
is list
1.0752737760627704
is list
1.0447356601699744
is list
1.0115635342248432
is list
0.8422404696131899


In [460]:
df_qa.to_json('/Users/jnaiman/Dropbox/JCDL2024/paper_submission/df_final.json', index=False)

In [458]:
# df_qa_one = df_qa[df_qa['file'].isin(df_qa[df_qa['correct answer'] == {'nrows':1,'ncols':1}]['file'].unique())].copy()
# len(df_qa),len(df_qa_one)

In [457]:
# for q in questions_one:
#     qt = list(q.keys())[0]
#     #accuracy_this_q = 0
#     accuracy_type = ''
#     values_correct = []; values_gpt = []
#     if type(q[qt]) == str: # just one question
#         d = df_qa_one[df_qa_one['question'] == q[qt]]
#         # what type
#         if len(d) == 0:
#             print('no matches for:', q[qt])
#             continue
#         if type(d.iloc[0]['correct answer']) == type({}):
#             isNum = True
#             for k,v in d.iloc[0]['correct answer'].items():
#                 try:
#                     x = v/1.0
#                 except:
#                     isNum = False
#             if isNum: # number
#                 accuracy_type = 'Numerical'
#                 for c,gpt in zip(d['correct answer'].values, d['gpt answer'].values):
#                     for k,v in c.items():
#                         values_correct.append(float(v))
#                         values_gpt.append(float(gpt[k]))

#                 #accuracy_this_q = np.sum(np.abs(np.array(values_correct)-np.array(values_gpt)))/np.sum(np.array(values_correct))
#                 accuracy_this_q = np.median(np.abs(np.array(values_correct)-np.array(values_gpt))/np.array(values_correct))
#                 q['Accuracy'] = accuracy_this_q
#                 q['Accuracy Type'] = 'Numerical'
#                 q['Correct Values'] = values_correct
#                 q['GPT Values'] = values_gpt
#             else:
#                 #print('not num!')
#                 isStr = False
#                 for k,v in d.iloc[0]['correct answer'].items():
#                     if type(v) == str:
#                         isStr = True
#                 if isStr:  
#                     #print('is string!')
#                     #import sys; sys.exit()
#                     accuracy_type = 'String'
#                     vcos = []
#                     vc = []; gv = []
#                     for c,gpt in zip(d['correct answer'].values, d['gpt answer'].values):
#                         for k,v in c.items():
#                             #print(v,gpt[k],cosine_sim(v,gpt[k]))
#                             #vcos.append(cosine_sim(v,gpt[k]))
#                             e = Levenshtein.distance(v,gpt[k])
#                             bigger = np.max([len(v),len(gpt)])
#                             #print(v,gpt[k],np.abs(1.0-e/bigger))
#                             vcos.append(np.abs(e/bigger))
#                             vc.append(v); gv.append(gpt[k])

#                     q['Accuracy'] = np.mean(vcos)
#                     q['Accuracy Type'] = 'String'
#                     q['Correct Values'] = vc
#                     q['GPT Values'] = gv
#                 else:
#                     # try list
#                     isList = False
#                     for k,v in d.iloc[0]['correct answer'].items():
#                         if type(v) == list:
#                             isList = True 
#                     if isList:
#                         #print('is list')
#                         vcos = []
#                         cos_type = ''
#                         vc = []; gc = []
#                         for c,gpt in zip(d['correct answer'].values, d['gpt answer'].values):
#                             for k,v in c.items():
#                                 cor_vec = []
#                                 g_vec = []
#                                 isStr1 = False; isNum1 = False
#                                 for cc in v:
#                                     cor_vec.append(cc)
#                                     try:
#                                         x = cc/1.0
#                                         isNum1 = True
#                                     except:
#                                         pass
#                                     if type(cc) == str:
#                                         isStr1 = True
#                                 if isStr1 and isNum1:
#                                     print('mixed types, ours!')
#                                     continue

#                                 isStr1 = False; isNum1 = False
#                                 for cc in gpt[k]:
#                                     g_vec.append(cc)
#                                     try:
#                                         x = cc/1.0
#                                         isNum1 = True
#                                     except:
#                                         pass
#                                     if type(cc) == str:
#                                         isStr1 = True
#                                 if isStr1 and isNum1:
#                                     print('mixed types, gpt!')
#                                     continue

#                                 if isStr1:
#                                     s1 = "".join(cor_vec)
#                                     s2 = "".join(g_vec)
#                                     e = Levenshtein.distance(s1,s2)
#                                     bigger = np.max([len(s1),len(s2)])
#                                     vcos.append(np.abs(e/bigger))   
#                                     #print(s1,s2, e/bigger)
#                                     if cos_type == '':
#                                         cos_type = ['String']
#                                     else: 
#                                         cos_type.append('String')
#                                     vc.append(s1); gc.append(s2)
#                                 else:
#                                     print('not string array!')
#                                     import sys; sys.exit()
                                
#                                 #cc = []; gg = []
#                                 #if len(v) != len(gpt[k]): # wrong items
#                                 #for cc,gg in zip(v,gpt[k]): # lists
#                         u = np.unique(cos_type)
#                         if len(u) > 1:
#                             print('too many cos types!')
#                             import sys; sys.exit()
#                         else:
#                             u = u[0]
#                         q['Accuracy'] = np.mean(vcos)
#                         q['Accuracy Type'] = u
#                         q['Correct Values'] = vc
#                         q['GPT Values'] = gc
#                         #import sys; sys.exit()
#                         #pass
#                     else:
                            
#                         print('not string or num or list!')
#                         import sys; sys.exit()
#         else:
#             print('not dict!')
#             import sys; sys.exit()
#     else: # this is for things that have multiple question types
#         #print('not a string!')
#         qout = []
#         print(qt)
#         for qq in q[qt]:
#             values_correct = []; values_gpt = []
#             vc = []; gc = []
#             d = df_qa_one[df_qa_one['question'] == qq]
#             # what type
#             if len(d) == 0:
#                 print('no matches for:', qq)
#                 continue
#             if type(d.iloc[0]['correct answer']) == type({}):
#                 isNum = True
#                 for k,v in d.iloc[0]['correct answer'].items():
#                     try:
#                         x = v/1.0
#                     except:
#                         isNum = False
#                 if isNum: # number
#                     accuracy_type = 'Numerical'
#                     for c,gpt in zip(d['correct answer'].values, d['gpt answer'].values):
#                         for k,v in c.items():
#                             values_correct.append(float(v))
#                             values_gpt.append(float(gpt[k]))
    
#                     #accuracy_this_q = np.sum(np.abs(np.array(values_correct)-np.array(values_gpt)))/np.sum(np.array(values_correct))
#                     accuracy_this_q = np.median(np.abs(np.array(values_correct)-np.array(values_gpt))/np.abs(np.array(values_correct)))
#                     print(accuracy_this_q)
#                     qout.append({'Accuracy': accuracy_this_q,'Accuracy Type': 'Numerical', 'Correct Values':values_correct, 'GPT Values':values_gpt})
#                 else:
#                     #print('not num!')
#                     isStr = False
#                     for k,v in d.iloc[0]['correct answer'].items():
#                         if type(v) == str:
#                             isStr = True
#                     if isStr:  
#                         #print('is string!')
#                         #import sys; sys.exit()
#                         accuracy_type = 'String'
#                         vcos = []
#                         vc = [];gc = []
#                         for c,gpt in zip(d['correct answer'].values, d['gpt answer'].values):
#                             for k,v in c.items():
#                                 #print(v,gpt[k],cosine_sim(v,gpt[k]))
#                                 #vcos.append(cosine_sim(v,gpt[k]))
#                                 e = Levenshtein.distance(v,gpt[k])
#                                 bigger = np.max([len(v),len(gpt)])
#                                 #print(v,gpt[k],np.abs(1.0-e/bigger))
#                                 vcos.append(np.abs(e/bigger))
#                                 vc.append(v);gc.append(gpt[k])
    
#                         qout.append({'Accuracy': np.mean(vcos),'Accuracy Type':'String', 'Correct Values':vc, 'GPT Values':gc})
#                     else:
#                         # try list
#                         isList = False
#                         for k,v in d.iloc[0]['correct answer'].items():
#                             if type(v) == list:
#                                 isList = True 
#                         if isList:
#                             print('is list')
#                             vcos = []
#                             cos_type = ''
#                             vc = []; gc = []
#                             for c,gpt in zip(d['correct answer'].values, d['gpt answer'].values):
#                                 vvc = []; ggc =[]
#                                 for k,v in c.items():
#                                     cor_vec = []
#                                     g_vec = []
#                                     isStr1 = False; isNum1 = False
#                                     for cc in v:
#                                         cor_vec.append(cc)
#                                         try:
#                                             x = cc/1.0
#                                             isNum1 = True
#                                         except:
#                                             pass
#                                         if type(cc) == str:
#                                             isStr1 = True
#                                     if isStr1 and isNum1:
#                                         print('mixed types, ours!')
#                                         continue
    
#                                     isStr1 = False; isNum1 = False
#                                     for cc in gpt[k]:
#                                         g_vec.append(cc)
#                                         try:
#                                             x = cc/1.0
#                                             isNum1 = True
#                                         except:
#                                             pass
#                                         if type(cc) == str:
#                                             isStr1 = True
#                                     if isStr1 and isNum1:
#                                         print('mixed types, gpt!')
#                                         continue
    
#                                     if isStr1:
#                                         s1 = "".join(cor_vec)
#                                         s2 = "".join(g_vec)
#                                         #print(s1,s2)
#                                         vvc.append(s1); ggc.append(s2)
#                                         e = Levenshtein.distance(s1,s2)
#                                         bigger = np.max([len(s1),len(s2)])
#                                         vcos.append(np.abs(e/bigger))   
#                                         #print(s1,s2, e/bigger)
#                                         if cos_type == '':
#                                             cos_type = ['String']
#                                         else: 
#                                             cos_type.append('String')
#                                         vvc.append(s1);ggc.append(s2)
#                                     elif isNum1:
#                                         try:
#                                             vcos.append(np.mean(np.abs(np.array(cor_vec)-np.array(g_vec))/np.abs(np.array(cor_vec))))
#                                         except:
#                                             vcos.append(np.mean(np.abs(np.mean(cor_vec)-np.mean(g_vec))/np.abs(np.array(cor_vec))))

#                                         vvc.append(cor_vec); ggc.append(g_vec)
#                                         if cos_type == '':
#                                             cos_type = ['Numerical']
#                                         else: 
#                                             cos_type.append('Numerical')

#                                     else:
#                                         print('not string or num array!')
#                                         import sys; sys.exit()
                                    
#                                     #cc = []; gg = []
#                                     #if len(v) != len(gpt[k]): # wrong items
#                                     #for cc,gg in zip(v,gpt[k]): # lists
#                                 vc.append(vvc); gc.append(ggc)
#                             u = np.unique(cos_type)
#                             if len(u) > 1:
#                                 print('too many cos types!')
#                                 import sys; sys.exit()
#                             else:
#                                 u = u[0]
#                             print(np.mean(vcos))
#                             qout.append({'Accuracy':np.mean(vcos),'Accuracy Type':u, 'Correct Values':vc, 'GPT Values':gc})
#                             #import sys; sys.exit()
#                             #pass
#                         else:
                                
#                             print('not string or num or list!')
#                             import sys; sys.exit()
#             else:
#                 print('not dict!')
#                 import sys; sys.exit()

#         atypes = []
#         accs = []
#         vc = []; gc = []
#         for qq in qout:
#             atypes.append(qq['Accuracy Type'])
#             accs.append(qq['Accuracy'])
#             vc.append(qq['Correct Values'])
#             gc.append(qq['GPT Values'])
#         u = np.unique(atypes)
#         if len(u) > 1:
#             print('mixed acc types!')
#             import sys; sys.exit()
#         else:
#             u = u[0]
#         q['Accuracy Type'] = u
#         q['Accuracy'] = np.median(accs)
#         #print(np.mean(accs))
#         q['Correct Values'] = vc
#         q['GPT Values'] = gc
#         #import sys; sys.exit()


In [461]:
for q in questions:
    if 'Accuracy' in q:
        print(list(q.keys())[0],':', q['Accuracy'], q['Accuracy Type'])

Number of panels : 0.0 Numerical
Plot style : 0.8525639034131093 String
Colormap : 1.2291666666666667 String
Aspect ratio : 0.15027216252972375 Numerical
Plot type : 0.26455025421090683 String
Number of scatter points : 0.7715678919729932 Numerical
Titles : 0.6470536552504235 String
x-axis titles : 0.2840952762883133 String
y-axis titles : 0.3613751902220772 String
x-y relationship : 0.48398268398268396 String
mean x/y : 0.9722869203528528 Numerical
median x/y : 1.0115635342248432 Numerical


In [452]:
for q in questions_one:
    if 'Accuracy' in q:
        print(list(q.keys())[0],':', q['Accuracy'], q['Accuracy Type'])

Number of panels : 0.0 Numerical
Plot style : 0.8279735311922405 String
Colormap : 1.2164179104477613 String
Aspect ratio : 0.055102569270172055 Numerical
Plot type : 0.19277090377836645 String
Number of scatter points : 0.3918918918918919 Numerical
Titles : 0.6623916908933984 String
x-axis titles : 0.14154619874791807 String
y-axis titles : 0.19906192777591497 String
x-y relationship : 0.6076438907821886 String
mean x/y : 0.8738000370049 Numerical
median x/y : 0.8767083056496962 Numerical


In [466]:
# # name, numerical error, string (edit distance), numerical error (n=1), string (n=1)
# print(r'\textbf{Metric} & \textbf{Numerical Error [\%]} & \textbf{String Error [e.d. \%]} & \textbf{Numerical Error, n=1 [\%]} & \textbf{String Error, n=1 [e.d. \%]} \\')
# for q,q1 in zip(questions,questions_one):
#     if q['Accuracy Type'] == 'Numerical':
#         print(list(q.keys())[0], '&', round(q['Accuracy']*100,2), '& -- &', round(q1['Accuracy']*100,2), '& -- \\\\')
#     else:
#         print(list(q.keys())[0], '& -- &', round(q['Accuracy']*100,2), '& -- &', round(q1['Accuracy']*100,2), '\\\\')


# name, numerical error, string (edit distance), numerical error (n=1), string (n=1)
print(r'\textbf{Metric} & \textbf{Numerical Error [\%]} & \textbf{String Error [e.d. \%]}  \\')
for q,q1 in zip(questions,questions_one):
    if q['Accuracy Type'] == 'Numerical':
        print(list(q.keys())[0], '&', round(q['Accuracy']*100,2), '& -- \\\\', '%', q['Level'])
    else:
        print(list(q.keys())[0], '& -- &', round(q['Accuracy']*100,2), ' \\\\', '%', q['Level'])

\textbf{Metric} & \textbf{Numerical Error [\%]} & \textbf{String Error [e.d. \%]}  \\
Number of panels & 0.0 & -- \\ % Level 1
Plot style & -- & 85.26  \\ % Level 1
Colormap & -- & 122.92  \\ % Level 1
Aspect ratio & 15.03 & -- \\ % Level 1
Plot type & -- & 26.46  \\ % Level 1
Number of scatter points & 77.16 & -- \\ % Level 1
Titles & -- & 64.71  \\ % Level 1
x-axis titles & -- & 28.41  \\ % Level 1
y-axis titles & -- & 36.14  \\ % Level 1
x-y relationship & -- & 48.4  \\ % Level 3
mean x/y & 97.23 & -- \\ % Level 2
median x/y & 101.16 & -- \\ % Level 2


In [468]:
j

[{'Q': 'How many panels are in this figure? You are a helpful assistant, please format the output as a json as {"nrows":"", "ncols":""} to store the number of rows and columns.',
  'A': {'nrows': 1, 'ncols': 1},
  'Level': 'Level 1',
  'type': 'Figure-level questions',
  'Response': {'nrows': '1', 'ncols': '1'},
  'Response String': '{"nrows":"1", "ncols":"1"}'},
 {'Q': 'Assuming this is a figure made with matplotlib in Python, what is the plot style used?  Examples of plotting styles are "classic" or "ggplot". You are a helpful assistant, please format the output as a json as {"plot style":""} to store the matplotlib plotting style used in the figure.',
  'A': {'plot style': '_mpl-gallery-nogrid'},
  'Level': 'Level 1',
  'type': 'Figure-level questions',
  'Response': {'plot style': 'seaborn-talk'},
  'Response String': '{"plot style":"seaborn-talk"}'},
 {'Q': 'Assuming this is a figure made with matplotlib in Python, what is the colormap that was used?  Examples of matplotlib colorm

In [464]:
#questions_one

In [465]:
#questions

In [317]:
for q in questions:
    if 'Accuracy' in q:
        print(q)
        print('')

{'Number of panels': 'How many panels are in this figure? You are a helpful assistant, please format the output as a json as {"nrows":"", "ncols":""} to store the number of rows and columns.', 'Accuracy': 0.11842105263157894, 'Accuracy Type': 'Numerical'}

{'Plot style': 'Assuming this is a figure made with matplotlib in Python, what is the plot style used?  Examples of plotting styles are "classic" or "ggplot". You are a helpful assistant, please format the output as a json as {"plot style":""} to store the matplotlib plotting style used in the figure.', 'Accuracy': 0.8525639034131093, 'Acurracy Type': 'String'}

{'Colormap': 'Assuming this is a figure made with matplotlib in Python, what is the colormap that was used?  Examples of matplotlib colormaps are "rainbow" or "Reds". You are a helpful assistant, please format the output as a json as {"colormap":""} to store the matplotlib colormap used in the figure.', 'Accuracy': 1.2291666666666667, 'Acurracy Type': 'String'}

{'Aspect rati

In [163]:
# aspect ratio -- how off
# colormap off/on
# plot stylle off/on/ how close?
# 'x' and 'y' for data -- how close
# median/max/min -- x/y -- how close
# linear/gmm relationship params
# x/y tick values -- how off/close? ==> maybe mean + std AND number?
# nlines/nbars
# plot types -- yes/no
# distribution type -- yes/no
# titles & x/y labels -- yes/no, maybe cosine sim?

38

In [174]:
json.loads(qa['Response'])

{'aspect ratio': '1.29'}